# Electric Field Calculations - 2

M&I 4e

Ch 13 Electric Field

- §13.5 &ndash; Superposition of Electric Fields
- §13.6 &ndash; The Electric Field of a Dipole

Ch 15 Electric Field of Distributed Charges

- §15.1 &ndash; A Uniformly Charged Thin Rod
- §15.2 &ndash; Procedure for Calculating Electric Field
- §15.4 &ndash; A Uniformly Charged Disk

In [ ]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]*</h2>

>   ---
>   ## [1]*
>
>   At a particular moment, three charged particles are located as shown below.
>
>   {{ graphic }}
>
>   $Q_1 = -5.5$ [µC], $Q_2 = 6.5$ [µC], $Q_3 = -8.0$ [µC]. 
>
>   Your answers to the following should be vectors. 
>
>   Recall that $1$ [µC] = $1 \times 10^{-6}$ [C].
>
>   Assume that the $+x$-axis is to the right, the $+y$-axis is up along the
>   page, and the $+z$-axis points out of the page. 
>
>   Express your answers in vector form. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='1a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Find the electric field at the location of $Q_3$, due to $Q_1$.__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E}_1 = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Find the electric field at the location of $Q_3$, due to $Q_2$.__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E}_2 = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(C)
>
>   __Find the net electric field at the location of $Q_3$.__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E}_\mathrm{net} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(D)
>
>   __Find the net force on $Q_3$.__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{F}_{\mathrm{net,3}} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(E)
>
>   __Find the electric field at Location $A$ due to $Q_1$.__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E}_{1} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1f' style="display:none">(F)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(F)
>
>   __Find the electric field at Location $A$ due to $Q_2$.__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E}_{2} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1g' style="display:none">(G)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(G)
>
>   __Find the electric field at Location $A$ due to $Q_3$.__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E}_{3} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1h' style="display:none">(H)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(H)
>
>   __What is the net the electric field at Location $A$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E}_{A} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1i' style="display:none">(I)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(I)
>
>   If a particle with charge $-6$ [nC] were placed at location $A$,
>   __what would be the force on this particle?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{F}_{\textrm{on}\ A} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]*</h2>

>   ---
>   ## [2]*
>
>   An $\mathrm{Fe^{3+}}$ ion is located $400$ [nm] ($400 \times 10^{-9}$ [m], 
>   about 4000 atomic diameters), from a $\mathrm{Cl^-}$ ion, as shown in the 
>   figure below (ions not shown to scale).
>
>   {{ graphic }}
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='2a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Determine the magnitude and direction of the electric field $\vec{E}_A$ at
>   location $A$__, $r=115$ [nm] to the __left__ of the $\mathrm{Cl^-}$ ion.
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{magnitude} = \fbox{\hspace{8em}?}\ \left[\mathrm{N/C}\right]$
>
>   $\textrm{direction} =$ 
>
>   (select one:)
>
>   - [ ]   to the right
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x]   to the right

<h3 id='2b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Determine the magnitude and direction of the electric field $\vec{E}_B$ at
>   location $B$__, $r=115$ [nm] to the __right__ of the $\mathrm{Cl^-}$ ion.
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{magnitude} = \fbox{\hspace{8em}?}\ \left[\mathrm{N/C}\right]$
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{direction} =$ 
>   
>   (select one:)
>
>   - [ ]   to the left
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x]   to the left

<h3 id='2c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(C)
>
>   If an electron is placed at Location $A$, 
>   __what are the magnitude and direction of the force on this electron?__
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{magnitude} = \fbox{\hspace{8em}?}\ \left[\mathrm{N}\right]$
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{direction} =$ 
>   
>   (select one:)
>
>   - [ ]   to the left
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x]   to the left

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]*</h2>

>   ---
>   ## [3]*
>
>   A plastic rod $1.6$ [m] long is rubbed all over with wool, and acquires a 
>   charge of $-8\mathrm{e}^-8$ coulombs.
>
>   We choose the center of the rod to be the origin of our coordinate system, 
>   with the $x$-axis extending to the right, the $y$-axis extending up, and the 
>   $z$-axis out of the page. 
>
>   In order to calculate the electric field at location $A = \langle\ 0.7, 0, 0\rangle$ [m],
>   we divide the rod into 8 pieces, and approximate each piece as a point charge
>   located at the center of the piece.
>
>   {{ graphic | graphic }}
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='3a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the length of one of these pieces?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the location of the center of piece number 5?__
>
>   &emsp;&emsp;&emsp;&emsp; $\left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(C)
>
>   __How much charge is on piece number 5?__
>   (Remember that the charge is negative).
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{coulombs}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(D)
>
>   Approximating piece number 5 as a point charge, __what is the electric field
>   at location $A$,__ due only to piece 5?
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E}_5 = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(E)
>
>   To get the net electric field at location $A$, we would need to calculate
>   the field due to each of the 8 pieces, and sum up these contributions.
>
>   If we did that, __which arrow above would best represent the direction of
>   the net electric field at location $A$?__
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] g

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]
>
>   An electrostatic dust precipitator that is installed in a factory smokestack
>   includes a straight metal wire of length $L = 0.8$ [m], that is charged with
>   approximately uniformly with a total charge $Q = 0.2 \times 10^{-7}$ [C].
>
>   A speck of coal dust (which is mostly carbon) is near the wire, and far from
>   both ends of the wire; the distance from the wire to the speck is $d = 1.4$ 
>   [cm]. Carbon has an atomic mass of 12 (6 protons and 6 neutrons in the 
>   nucleus). A careful measurement of the polarizability of a carbon atom gives
>   the value $\alpha = 1.96 \times 10^{-40}$ [(C⋅m)/(N/C)]. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Calculate the initial acceleration of the speck of coal dust, 
>   neglecting gravity.__
>
>   Your answer must be expressed in terms of $Q$, $L$, $d$, and $\alpha$.
>   You can use other quantities in your calculations, but your final result
>   must not include them. 
>
>   It is convenient to use the "binomial expansion", that you may have learned
>   in calculus, that $(1 + \epsilon)^n \approx 1 + n\epsilon$, if $\epsilon \ll 1$.
>   Note that $n$ can be negative. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h4 id='4a_i' style="display:none">{A.ⅰ}</h4>

>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>
>   
>   #### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;{A.ⅰ}
>
>   __Algebraically express the magnitude of the acceleration $a$.__
>
>   Use the following as necessary: $Q$, $L$, $d$, $\alpha$, $\epsilon_0$, and
>   $m$ for the mass of a carbon atom. Use the binomial expansion in your answer.
>
>   (select one:)
>
>   - [ ] &nbsp; $a = \left(\frac{1}{4\pi\epsilon_0}\right)^2 \frac{4 Q^2 \alpha}{m L^2 d^3}$
>
>   - [ ] &nbsp; $a = \left(\frac{1}{4\pi\epsilon_0}\right)^2 \frac{Q^2 \alpha}{m L^2 d^3}$
>
>   - [ ] &nbsp; $a = \left(\frac{1}{4\pi\epsilon_0}\right)^2 \frac{4 Q^2 \alpha}{m L^3 d^2}$
>
>   - [ ] &nbsp; $a = \left(\frac{1}{4\pi\epsilon_0}\right)^2 \frac{Q^2 \alpha}{m L^3 d^2}$
>
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<h4 id='4a_ii' style="display:none">{A.ⅱ}</h4>

>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>
>   
>   #### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;{A.ⅱ}
>
>   __Calculate the initial acceleration of the speck of coal dust, neglecting
>   gravity.__
>
>   Don't put numbers into your calculation until the very end, but then show
>   the numerical calculation that you carry out on your calculator.
>
>   (Enter the magnitude:)
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{m / s^2}\right]$
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<h3 id='10b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(B)
>
>   If the speck of coal dust were initially five times as far from the charged
>   wire, __how much smaller would the initial acceleration of the speck be?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\frac{a_{5d}}{a_d} = \fbox{\hspace{8em}?}$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]*</h2>

>   ---
>   ## [5]*
>
>   A strip of invisible tape, $0.19$ [m] long by $0.015$ [m] wide, is uniformly
>   charged with a total net charge of $4$ [nC] (nano `n` = $1 \times 10^{-9}$);
>   and is suspended horizontally so it lies along the $x$-axis, with its center
>   at the origin (as shown in the figure below). 
>
>   {{ graphic }}
>
>   Calculate the approximate electric field at location A, 
>   $\left\langle\ 0, 0.02, 0\right\rangle$ [m], due to the strip of tape. 
>
>   Do this by dividing the strip into three equal sections, as in the figure 
>   above, and approximating each section as a point charge. (Assume each point
>   charge is located at the center of the section it approximates.)
>
>   Express your answers in vector form. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='5a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the approximate electric field at $A$, due to piece 1?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_1 =  \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the approximate electric field at $A$, due to piece 2?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_2 =  \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the approximate electric field at $A$, due to piece 3?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_3 =  \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the approximate _net_ electric field at $A$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\vec{E}_3 =  \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What could you do to improve the accuracy of your calculations?__
>
>   (select one:)
>
>   - [ ] &nbsp; Use a _larger_ charge for piece 2, than for pieces 1 and 3:
>                since location $A$ is closest to piece 2, this piece has the 
>                most significant contribution. 
>
>   - [ ] &nbsp; Use more sections: as the number of segments approaches infinity,
>                the summation of the electric fields approximates the actual 
>                electric field. 
>
>   - [ ] &nbsp; Use a _smaller_ charge for piece 2, than for pieces 1 and 3:
>                the actual electric field is similar to that of a dipole, 
>                with most of the charge on the tape at the left or right ends.
>
>   - [ ] &nbsp; Use just two sections: due to symmetry about the $y$-axis, 
>                the components of the field due to each charge will be half of
>                the total electric field. 
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] &nbsp; Use more sections: as the number of segments approaches infinity,
             the summation of the electric fields approximates the actual 
             electric field. 

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [6]
>
>   For a disk of radius $R = 24$ [cm] and charge $Q = 5 \times 10^{-6}$ [C],
>   calculate the electric field $4$ [cm] from the center of the disk using all
>   three formulas below.
>
>   Note: $\epsilon_0 = 8.85\mathrm{e}^-12$ [C²/(N⋅m²)]
>
>   {{ graphic }}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Most accurate approximation:__ 
>   $E = \frac{Q/A}{2\epsilon_0}\left[ 1 - \frac{z}{(R^2 + z^2)^{1/2} }\right]$
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}$&emsp;[N/C]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Less accurate approximation:__ 
>   $E = \frac{Q/A}{2\epsilon_0}\left[ 1 - \frac{z}{R} \right]$
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}$&emsp;[N/C]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(C)
>
>   __Least accurate approximation:__ 
>   $E = \frac{Q/A}{2\epsilon_0}$
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}$&emsp;[N/C]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(D)
>
>   __At what distance__ does the least approximation for the electric field
>   give a result that is closest to the most accurate?
>
>   (select one:)
>
>   - [ ]   at a distance $R$
>
>   - [ ]   close to the disk
>
>   - [ ]   far from the disk
>
>   - [ ]   at a distance $R/2$
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] close to the disk

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]</h2>

>   ---
>   ## [7]
>
>   The dipole moment of the $\mathrm{HBr}$ (hydrogen bromide) molecule has been
>   measured to be $2.74 \times 10^{-30}$ [C⋅m].
>
>   If we model the dipole as having charges of $+e$ and $-e$ separated by a 
>   distance $s$, __what is $s$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $s = \fbox{\hspace{8em}?}$&emsp;[pm]
>
>   __Is this plausible?__ 
>   (Assume this is plausible if the separation distance $s$ is greater than 
>   $100$ [pm], the approximate diameter of a hydrogen atom.)
>
>   (select one:)
>
>   - [ ] Yes
>   - [ ] No
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

- [x] No

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   ---
>   ## [8]
>
>   A dipole consists of two charges, $+q$ and $-q$, held apart by a rod of 
>   length $s$; as shown in the diagram. 
>
>   {{ graphic }}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='8a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(A)
>
>   If $q = 7$ [nC] and $s=3$ [mm], __what is the magnitude of the electric field due to the dipole at location $A$,__
>   a distance $d=5$ [cm] from the dipole?
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $E = \fbox{\hspace{8em}?}$&emsp;[N/C]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='8b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the magnitude of the electric field due to the dipole at location $B$,__
>   a distance $d=5$ [cm] from the dipole?
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $E = \fbox{\hspace{8em}?}$&emsp;[N/C]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='9' style="display:none">[9]</h2>

>   ---
>   ## [9]
>
>   Two identical permanent dipoles, each consisting of charges $+q$ and $-q$
>   separated by a distance $s$; are aligned on the $x$-axis, a distance $r$ 
>   from each other, where $r \gg s$ (see figure below).
>
>   {{ graphic }}
>
>   __What is the magnitude of the net force exerted on one dipole, by the other dipole?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $F \approx \fbox{\hspace{6em}?}$&emsp;[N]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

---